In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from numpy.linalg import inv,qr,solve,lstsq
from mediation_simulation import *
from linReg import *

In [3]:
np.random.seed(0)

In [4]:
var={'n':4,
     'p_otu':8, 
     'p_metabolite':4, 
     'mediations':1,
     'mediation_func':simple_mediation_direct
    }

In [5]:
truth_coef={
    'beta11':-2,
    'beta21':2,
    'beta32':-1.5
}

<hr>

# simple mediation direct
<hr>

In [137]:
np.random.seed(0)
output1=simulate_simple_mediation(**var)

In [138]:
output1["OTU"]

matrix([[ 0.5488135 ,  0.71518937,  0.60276338,  0.54488318],
        [ 0.4236548 ,  0.64589411,  0.43758721,  0.891773  ],
        [ 0.96366276,  0.38344152,  0.79172504,  0.52889492],
        [ 0.56804456,  0.92559664,  0.07103606,  0.0871293 ],
        [ 0.0202184 ,  0.83261985,  0.77815675,  0.87001215],
        [ 0.97861834,  0.79915856,  0.46147936,  0.78052918],
        [ 0.61209572,  0.616934  ,  0.94374808,  0.6818203 ],
        [ 0.72568213,  2.32277256, -2.69670283, -2.5822871 ]])

In [140]:
P,t=output1["OTU"][:-1,:],output1["OTU"][-1,:]

<hr>

# 1. $$\text{target}=\beta_{32}\cdot(\beta_{21}\text{producer}+\epsilon_1)+\epsilon_2$$

In [133]:
# solving for M=beta21.P+epsilon2
P=output1["OTU"][:-1]
P_1=np.vstack([P,np.ones(P.shape[1])]).T
coef,residuals,_,_=lstsq(P_1,t.T,rcond=None)

In [135]:
B,ep=coef[:-1],coef[-1]
display(B,ep)

matrix([[ 0.66138033],
        [-1.28352585],
        [-1.2494109 ],
        [ 4.99382612],
        [-0.82033155],
        [ 0.37055613],
        [-1.0843909 ]])

matrix([[-0.40852165]])

In [136]:
# truth value for beta21, estimated value for beta21
truth_coef["beta21"],B[-1,0]

(2, -1.0843908961055417)

In [128]:
# solving for T=beta32.M+epsilon1

M=output1["Metabolite"]
t=output1["OTU"][-1]
M_1=np.vstack([M,np.ones(M.shape[1])]).T
M_1

matrix([[ 0.11827443,  0.52184832,  0.45615033,  0.3364057 ,  1.        ],
        [ 0.63992102,  0.41466194,  0.56843395, -0.74692847,  1.        ],
        [ 0.14335329,  0.26455561,  0.0187898 ,  1.53958401,  1.        ],
        [ 0.94466892,  0.77423369,  0.6176355 ,  1.51998957,  1.        ]])

In [103]:
coef,residuals,_,_=lstsq(M_1,t.T,rcond=None)

In [123]:
B,ep=coef[:-1],coef[-1]
display(B,ep)

matrix([[-1.46321826],
        [ 1.03756947],
        [ 1.19532053],
        [-2.15757217]])

matrix([[0.53786326]])

In [122]:
# truth value for beta32, estimated value for beta32
truth_coef["beta32"],B[-1,0]

(-1.5, -2.157572165021854)

## $$\text{target}=\beta_{32}\beta_{21}\text{producer}+\beta_{32}\epsilon_{1}+\epsilon_2$$
## $\Rightarrow\enspace \text{let Y = target, } \beta_{10}=\beta{32}\epsilon_1, \beta_{11}=\beta_{32}\beta_{21}$
## $\Rightarrow\enspace Y=\beta_{10}+\beta_{11}X+\epsilon_{1}$

In [ ]:
# TODO: 

<hr>
<hr>

# 2

## step 1: $$Y=\beta_{10}+\beta_{11}X+\epsilon_{1}$$

In [10]:
# add dummy variable for beta10
X_1=np.vstack([X,np.ones(X.shape[1])])
X_1.shape

(8, 4)

In [11]:
coef, vcov, sigma,df=linmodEst(X_1,y)

In [12]:
display(sigma,coef)

9.496788308671569e-15

matrix([[ 0.66138033, -1.28352585, -1.2494109 ,  4.99382612, -0.82033155,
          0.37055613, -1.0843909 , -0.40852165]])

In [13]:
beta11,beta10=coef[:,:-1],coef[:,-1]

In [14]:
beta11.dot(X)+beta10

matrix([[ 0.72568213,  2.32277256, -2.69670283, -2.5822871 ]])

## step 2: $$Me=\beta_{20}+\beta_{21}X+\epsilon_{2}$$

In [15]:
coef2, vcov2, sigma2,df2=linmodEst(X_1,output["Metabolite"])

In [16]:
display(sigma2)

3.755217577292072e-15

In [17]:
beta21,beta20=coef2[:,:-1],coef2[:,-1]

In [18]:
beta21.dot(X)+beta20

matrix([[ 0.11827443,  0.63992102,  0.14335329,  0.94466892],
        [ 0.52184832,  0.41466194,  0.26455561,  0.77423369],
        [ 0.45615033,  0.56843395,  0.0187898 ,  0.6176355 ],
        [ 0.3364057 , -0.74692847,  1.53958401,  1.51998957]])

## step 3: $$Y=\beta_{30}+\beta_{31}X+\beta_{32}Me+\epsilon_{3}$$

In [19]:
XMe1=np.vstack([X,output["Metabolite"],np.ones(X.shape[1])])

In [20]:
coef3, vcov3, sigma3,df3=linmodEst(XMe1,y)

In [21]:
display(sigma3,coef3)

1.5627153193985828e-15

matrix([[ 0.13996157, -0.09131076,  0.19973646,  0.81445606, -0.49267889,
          0.46521756, -0.12846847, -0.1950714 ,  0.03303968,  0.28434894,
         -1.82447155,  0.15564041]])

In [22]:
beta30,beta31,beta32=coef3[:,-1],coef3[:,:X.shape[0]],coef3[:,X.shape[0]:-1]

In [23]:
display(beta30,beta31,beta32)

matrix([[0.15564041]])

matrix([[ 0.13996157, -0.09131076,  0.19973646,  0.81445606, -0.49267889,
          0.46521756, -0.12846847]])

matrix([[-0.1950714 ,  0.03303968,  0.28434894, -1.82447155]])

In [24]:
beta30+beta31.dot(X)+beta32.dot(output["Metabolite"])

matrix([[ 0.72568213,  2.32277256, -2.69670283, -2.5822871 ]])

<hr>

In [25]:
np.abs(beta31)<np.abs(beta11)

matrix([[ True,  True,  True,  True,  True, False,  True]])

In [28]:
beta31[:,-2],beta11[:,-2]

(matrix([[0.46521756]]), matrix([[0.37055613]]))